# Generated by alkh

In [1]:
import sys
sys.path.insert(0, r'/mnt/hgfs/dev/open_source_projects/alkh/tests')
sys.path.insert(1, r'/snap/pycharm-professional/306/plugins/python/helpers/pydev')
sys.path.insert(2, r'/snap/pycharm-professional/306/plugins/python/helpers/third_party/thriftpy')
sys.path.insert(3, r'/snap/pycharm-professional/306/plugins/python/helpers/pydev')
sys.path.insert(4, r'/mnt/dev/open_source_projects/alkh')
sys.path.insert(5, r'/mnt/hgfs/dev/open_source_projects/alkh/tests')
sys.path.insert(6, r'/mnt/hgfs/dev/azure_projects/ATool/research')
sys.path.insert(7, r'/snap/pycharm-professional/306/plugins/python/helpers/pycharm_display')
sys.path.insert(8, r'/home/erez/.cache/JetBrains/PyCharm2022.2/cythonExtensions')
sys.path.insert(9, r'/mnt/dev/open_source_projects/alkh/tests')
sys.path.insert(10, r'/home/erez/anaconda3/lib/python38.zip')
sys.path.insert(11, r'/home/erez/anaconda3/lib/python3.8')
sys.path.insert(12, r'/home/erez/anaconda3/lib/python3.8/lib-dynload')
sys.path.insert(13, r'/home/erez/.local/lib/python3.8/site-packages')
sys.path.insert(14, r'/home/erez/anaconda3/lib/python3.8/site-packages')
sys.path.insert(15, r'/snap/pycharm-professional/306/plugins/python/helpers/pycharm_matplotlib_backend')

In [2]:
import pandas as pd
import alkh

In [3]:
import networkx as nx
from alkh.cst_utils import CallGraphManager
import numpy as np
from typing import *
import libcst as cst

In [4]:
stack_df = pd.read_pickle('b62a6a2fb1664a6ca253201f8d420495.pkl')
stack_df[["file_path", "function", "lineno", "locals_names"]]

,file_path,function,lineno,locals_names
0,/mnt/dev/open_source_projects/alkh/alkh/cst_ut...,_calc_assignment_df,70,"[self, assignment_df]"
1,/mnt/dev/open_source_projects/alkh/alkh/cst_ut...,__init__,28,"[self, file_path]"
2,/mnt/dev/open_source_projects/alkh/tests/cst_u...,<module>,5,"[file_path, line_number]"


In [5]:
alkh.print_context(stack_df.loc[0, 'context'])

/mnt/dev/open_source_projects/alkh/alkh/cst_utils.py
67:         assignment_df["scope_index"] = assignment_df["line"].apply(self._get_scope_index, args=(self._scopes_df,))
68:         assignment_df['canonic_targets'] = assignment_df.apply(self._get_canonic_target_list, axis=1)
69:         self._assignment_df = assignment_df
70: 
71:     @staticmethod


In [6]:
self = stack_df.loc[0, 'locals']['self']
assignment_df = stack_df.loc[0, 'locals']['assignment_df']

In [7]:
class_scopes_df = self._class_scopes_df
scopes_df = self._scopes_df

In [ ]:
def _get_scope_index(line_number, scopes_df):
    query_string = f"start_line_number <= {line_number} and end_line_number >= {line_number}"
    relevant_scoped_df = scopes_df.query(query_string).sort_values("length")
    scope_index = relevant_scoped_df.iloc[0]['scope_index']
    return scope_index

In [ ]:
a = assignment_df["canonic_targets"].explode()
a

In [ ]:
b = pd.DataFrame(a.tolist())
b

In [ ]:
b["main_id"] = b["id"].apply(lambda x: x[1])

In [ ]:
b.head()

In [ ]:
c = b.groupby(["scope_index", "main_id"])["line_number"].apply(list).reset_index()
c

In [ ]:
d = c.groupby("scope_index")["main_id"].apply(tuple).reset_index()
d

In [ ]:
e = scopes_df.merge(d)
e

In [ ]:
def _get_canonic_source_list(assignment_series):
    scope_index = assignment_series['scope_index']
    line_number = assignment_series['line']

    canonic_source_list = []
    for source in assignment_series['sources']:
        canonic_source = _get_canonic_source(source, line_number, scope_index)
        if canonic_source is not None:
            canonic_source_list.append(canonic_source)

    return canonic_source_list

In [ ]:
def _get_canonic_source(source, line_number, scope_index):
#     print(source, line_number, scope_index)
    if source[0] == 'self':
        class_scope_index = _get_scope_index(line_number, class_scopes_df)
        canonic_source = tuple([class_scope_index] + source[1:])
    else:
        query_string = f"start_line_number <= {line_number} and end_line_number >= {line_number}"
        all_scopes_df = e.query(query_string).sort_values("length")
        contains_source = all_scopes_df["main_id"].apply(lambda x: source[0] in x)
        contains_source_scopes_df = all_scopes_df[contains_source]
        if not contains_source_scopes_df.empty:
            source_scope_index = contains_source_scopes_df.iloc[0]["scope_index"]
            canonic_source = tuple([source_scope_index] + source)
        else:
            canonic_source = None
    return canonic_source

In [ ]:
assignment_df["canonic_sources"] = assignment_df.apply(_get_canonic_source_list, axis=1)

In [ ]:
assignment_df

In [ ]:
assignment_df["canonic_targets_ids"] = assignment_df["canonic_targets"].apply(lambda x: [y["id"] for y in x])

In [ ]:
assignment_df

In [29]:
def _get_scope_index(line_number, scopes_df):
    query_string = f"start_line_number <= {line_number} and end_line_number >= {line_number}"
    relevant_scoped_df = scopes_df.query(query_string).sort_values("length")
    scope_index = relevant_scoped_df.iloc[0]['scope_index']
    return scope_index


def _get_canonic_source_list(assignment_series, scope_to_targets_df):
    line_number = assignment_series['line']

    canonic_source_list = []
    for source in assignment_series['sources']:
#         canonic_source = None
        canonic_source = _get_canonic_source(source, line_number, scope_to_targets_df)
        if canonic_source is not None:
            canonic_source_list.append(canonic_source)

    return canonic_source_list

def _get_canonic_source(source, line_number, scope_to_targets_df):
    if source[0] == 'self':
        class_scope_index = _get_scope_index(line_number, class_scopes_df)
        canonic_source = tuple([class_scope_index] + source[1:])
    else:
        query_string = f"start_line_number <= {line_number} and end_line_number >= {line_number}"
        all_scopes_df = scope_to_targets_df.query(query_string).sort_values("length")
        contains_source = all_scopes_df["main_id"].apply(lambda x: source[0] in x)
        contains_source_scopes_df = all_scopes_df[contains_source]
        if not contains_source_scopes_df.empty:
            source_scope_index = contains_source_scopes_df.iloc[0]["scope_index"]
            canonic_source = tuple([source_scope_index] + source)
        else:
            canonic_source = None
    return canonic_source

In [30]:
a = assignment_df["canonic_targets"].explode()
b = pd.DataFrame(a.tolist())
b["main_id"] = b["id"].apply(lambda x: x[1])
c = b.groupby(["scope_index", "main_id"])["line_number"].apply(list).reset_index()
d = c.groupby("scope_index")["main_id"].apply(tuple).reset_index()
scope_to_targets_df = scopes_df.merge(d)
assignment_df["canonic_sources"] = assignment_df.apply(_get_canonic_source_list, args=(scope_to_targets_df,), axis=1)
assignment_df["canonic_targets_ids"] = assignment_df["canonic_targets"].apply(lambda x: [y["id"] for y in x])

In [31]:
assignment_df

,targets,data,line,sources,scope_index,canonic_targets,canonic_sources,canonic_targets_ids
0,[[jj]],"{'names': [], 'ints': ['20'], 'floats': []}",6,[],0,"[{'self': False, 'scope_index': 0, 'id': (0, '...",[],"[(0, jj)]"
1,[[m]],"{'names': [], 'ints': ['200'], 'floats': []}",7,[],0,"[{'self': False, 'scope_index': 0, 'id': (0, '...",[],"[(0, m)]"
2,[[k]],"{'names': [], 'ints': ['8'], 'floats': []}",11,[],2,"[{'self': False, 'scope_index': 2, 'id': (2, '...",[],"[(2, k)]"
3,[[m]],"{'names': [['pd', 'Series']], 'ints': ['20'], ...",12,"[[pd, Series]]",2,"[{'self': False, 'scope_index': 2, 'id': (2, '...",[],"[(2, m)]"
4,"[[m, c]]","{'names': [], 'ints': ['50'], 'floats': []}",13,[],2,"[{'self': False, 'scope_index': 2, 'id': (2, '...",[],"[(2, m, c)]"
5,[[ll]],"{'names': [], 'ints': ['300'], 'floats': []}",14,[],2,"[{'self': False, 'scope_index': 2, 'id': (2, '...",[],"[(2, ll)]"
6,"[[self, m, c]]","{'names': [], 'ints': ['60'], 'floats': []}",17,[],3,"[{'self': True, 'scope_index': 2, 'id': (2, 'm...",[],"[(2, m, c)]"
7,"[[self, ll]]","{'names': [], 'ints': ['400'], 'floats': []}",18,[],3,"[{'self': True, 'scope_index': 2, 'id': (2, 'l...",[],"[(2, ll)]"
8,"[[self, k], [ll]]","{'names': [['jj']], 'ints': ['9', '10'], 'floa...",19,[[jj]],3,"[{'self': True, 'scope_index': 2, 'id': (2, 'k...","[(0, jj)]","[(2, k), (3, ll)]"
9,"[[b], [mm]]","{'names': [['self', 'k'], ['self', 'm', 'c'], ...",20,"[[self, k], [self, m, c], [self, k]]",3,"[{'self': False, 'scope_index': 3, 'id': (3, '...","[(2, k), (2, m, c), (2, k)]","[(3, b), (3, mm)]"


In [11]:
e

,scope,start_line_number,end_line_number,header_end_line_number,length,name,scope_index,main_id
0,<libcst.metadata.scope_provider.GlobalScope ob...,1,49,1,49,global,0,"(a, jj, m)"
1,<libcst.metadata.scope_provider.FunctionScope ...,41,46,41,6,run,1,"(a, b, c, d, ll)"
2,<libcst.metadata.scope_provider.ClassScope obj...,10,33,10,24,A,2,"(k, ll, m)"
3,<libcst.metadata.scope_provider.FunctionScope ...,16,21,16,6,__init__,3,"(b, ll, mm)"
4,<libcst.metadata.scope_provider.FunctionScope ...,23,33,26,11,run,4,"(a, b, c, d, k, ll)"
5,<libcst.metadata.scope_provider.FunctionScope ...,37,39,37,3,__init__,6,"(b,)"


In [26]:
c

,scope_index,main_id,line_number
0,0,a,[49]
1,0,jj,[6]
2,0,m,[7]
3,1,a,[42]
4,1,b,[43]
5,1,c,[45]
6,1,d,[46]
7,1,ll,[44]
8,2,k,"[11, 19]"
9,2,ll,"[14, 18]"


In [27]:
d

,scope_index,main_id
0,0,"(a, jj, m)"
1,1,"(a, b, c, d, ll)"
2,2,"(k, ll, m)"
3,3,"(b, ll, mm)"
4,4,"(a, b, c, d, k, ll)"
5,6,"(b,)"


In [28]:
scope_to_targets_df

,scope,start_line_number,end_line_number,header_end_line_number,length,name,scope_index,main_id
0,<libcst.metadata.scope_provider.GlobalScope ob...,1,49,1,49,global,0,"(a, jj, m)"
1,<libcst.metadata.scope_provider.FunctionScope ...,41,46,41,6,run,1,"(a, b, c, d, ll)"
2,<libcst.metadata.scope_provider.ClassScope obj...,10,33,10,24,A,2,"(k, ll, m)"
3,<libcst.metadata.scope_provider.FunctionScope ...,16,21,16,6,__init__,3,"(b, ll, mm)"
4,<libcst.metadata.scope_provider.FunctionScope ...,23,33,26,11,run,4,"(a, b, c, d, k, ll)"
5,<libcst.metadata.scope_provider.FunctionScope ...,37,39,37,3,__init__,6,"(b,)"


In [32]:
b

,self,scope_index,id,line_number,main_id
0,False,0,"(0, jj)",6,jj
1,False,0,"(0, m)",7,m
2,False,2,"(2, k)",11,k
3,False,2,"(2, m)",12,m
4,False,2,"(2, m, c)",13,m
5,False,2,"(2, ll)",14,ll
6,True,2,"(2, m, c)",17,m
7,True,2,"(2, ll)",18,ll
8,True,2,"(2, k)",19,k
9,False,3,"(3, ll)",19,ll


In [33]:
c

,scope_index,main_id,line_number
0,0,a,[49]
1,0,jj,[6]
2,0,m,[7]
3,1,a,[42]
4,1,b,[43]
5,1,c,[45]
6,1,d,[46]
7,1,ll,[44]
8,2,k,"[11, 19]"
9,2,ll,"[14, 18]"


In [34]:
d

,scope_index,main_id
0,0,"(a, jj, m)"
1,1,"(a, b, c, d, ll)"
2,2,"(k, ll, m)"
3,3,"(b, ll, mm)"
4,4,"(a, b, c, d, k, ll)"
5,6,"(b,)"


In [36]:
c.head()

,scope_index,main_id,line_number
0,0,a,[49]
1,0,jj,[6]
2,0,m,[7]
3,1,a,[42]
4,1,b,[43]


In [40]:
c['target_id'] = c.apply(lambda x: (x['scope_index'], x['main_id']), axis=1)

In [47]:
target_id_to_line_numbers_df = c[['target_id', 'line_number']].rename(columns={'line_number': 'line_numbers_list'}).copy()
target_id_to_line_numbers_df

,target_id,line_numbers_list
0,"(0, a)",[49]
1,"(0, jj)",[6]
2,"(0, m)",[7]
3,"(1, a)",[42]
4,"(1, b)",[43]
5,"(1, c)",[45]
6,"(1, d)",[46]
7,"(1, ll)",[44]
8,"(2, k)","[11, 19]"
9,"(2, ll)","[14, 18]"
